In [1]:
import librosa
import numpy as np
import os
import math
import operator
from sklearn.cluster import KMeans
import hmmlearn.hmm
import functionMFCC as func

C:\Users\PhuongDong\miniconda3\envs\voice\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
C:\Users\PhuongDong\miniconda3\envs\voice\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [38]:
class_names = ["nguoi", "viet_nam", "nha", "cua", "khong", "test_nguoi", "test_viet_nam", "test_nha", "test_cua", "test_khong"]
dataset = {}
for cname in class_names:
    print(f"Load {cname} dataset")
    dataset[cname] = func.get_class_data(os.path.join("data_hmm", cname))
# Get all vectors in the datasets
all_vectors = np.concatenate([np.concatenate(v, axis=0) for k, v in dataset.items()], axis=0)
# print("vectors", all_vectors.shape)
# # Run K-Means algorithm to get clusters
# kmeans = func.clustering(all_vectors)
# print("centers", kmeans.cluster_centers_.shape)
# for cname in class_names:
#     dataset[cname] = list([kmeans.predict(v).reshape(-1,1) for v in dataset[cname]])

Load nguoi dataset
Load viet_nam dataset
Load nha dataset
Load cua dataset
Load khong dataset
Load test_nguoi dataset
Load test_viet_nam dataset
Load test_nha dataset
Load test_cua dataset
Load test_khong dataset


In [60]:
def train_model(train_set, n_component, startprob):
    model = hmmlearn.hmm.GMMHMM(n_components=n_component, n_mix=3, n_iter=1000, params='mtcw', init_params='mt', verbose=True)
    model.startprob_ = startprob
    X = np.concatenate(train_set)
    lengths = list([len(x) for x in train_set])
    # fit dat 
    print(X.shape, lengths, len(lengths))
    model.fit(X)
    return model

In [62]:
startprob9 = np.array([0.6, 0.3, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
startprob18 = np.array([0.6, 0.3, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
# transmat6 = np.array([[0.05 for i in range(6)] for j in range(6)])
# transmat12 = np.array([[0.05 for i in range(12)] for j in range(12)])
# for x in range(5):
#     transmat6[x][x] = 0.4
#     transmat6[x][x+1] = 0.4
# transmat6[5][5] = 0.75
# for x in range(11):
#     transmat12[x][x] = 0.25
#     transmat12[x][x+1] = 0.25
# transmat12[11][11] = 0.45
# print(transmat6)
models = {}
models["khong"] = train_model(dataset["khong"], 9, startprob9)
models["nha"] = train_model(dataset["nha"], 9, startprob9)
models["cua"] = train_model(dataset["cua"], 9, startprob9)
models["nguoi"] = train_model(dataset["nguoi"], 9, startprob9)
models["viet_nam"] = train_model(dataset["viet_nam"], 18, startprob18)

(1887, 36) [25, 21, 24, 27, 25, 21, 18, 24, 24, 21, 24, 29, 15, 31, 19, 21, 20, 25, 29, 32, 31, 23, 18, 35, 23, 21, 24, 28, 33, 28, 34, 23, 17, 17, 17, 23, 25, 25, 22, 25, 28, 22, 24, 19, 17, 21, 19, 23, 19, 35, 26, 24, 27, 21, 20, 23, 20, 21, 24, 16, 25, 19, 26, 21, 22, 26, 27, 19, 17, 35, 28, 22, 25, 29, 31, 21, 19, 21, 13, 25] 80


         1     -193792.2844             +nan
         2     -179715.4734      +14076.8110
         3     -177470.5110       +2244.9624
         4     -176820.0121        +650.4989
         5     -176513.1929        +306.8192
         6     -176356.8342        +156.3587
         7     -176210.9539        +145.8802
         8     -176123.1810         +87.7729
         9     -176044.0171         +79.1639
        10     -175959.6875         +84.3296
        11     -175894.7605         +64.9271
        12     -175841.3403         +53.4202
        13     -175789.9186         +51.4217
        14     -175719.2010         +70.7176
        15     -175615.9156        +103.2854
        16     -175580.4454         +35.4702
        17     -175558.2898         +22.1556
        18     -175487.8182         +70.4716
        19     -175448.2655         +39.5527
        20     -175425.4734         +22.7921
        21     -175406.3575         +19.1159
        22     -175378.8156         +27.5418
        23

(2541, 36) [21, 22, 30, 23, 17, 23, 28, 47, 32, 24, 42, 32, 33, 36, 25, 20, 25, 45, 22, 31, 28, 17, 42, 27, 19, 20, 33, 16, 36, 31, 39, 17, 33, 31, 28, 35, 32, 38, 18, 43, 29, 25, 22, 25, 21, 24, 32, 23, 62, 33, 22, 18, 33, 22, 16, 18, 54, 46, 52, 45, 22, 40, 32, 25, 26, 41, 32, 43, 43, 43, 26, 59, 40, 45, 32, 50, 47, 59, 32, 21] 80


         1     -245327.9204             +nan
         2     -225553.6594      +19774.2610
         3     -196677.2365      +28876.4229
         4     -181399.0808      +15278.1557
         5     -180917.3351        +481.7456
         6     -180751.4482        +165.8870
         7     -180657.7391         +93.7090
         8     -180549.5081        +108.2311
         9     -180450.1176         +99.3905
        10     -180392.4548         +57.6628
        11     -180347.2005         +45.2543
        12     -180306.0423         +41.1582
        13     -180257.3576         +48.6847
        14     -180240.1991         +17.1585
        15     -180217.1474         +23.0517
        16     -180201.0521         +16.0953
        17     -180195.2990          +5.7531
        18     -180189.0359          +6.2632
        19     -180183.8562          +5.1797
        20     -180179.6117          +4.2446
        21     -180177.5646          +2.0471
        22     -180175.6959          +1.8687
        23

(1872, 36) [55, 18, 24, 23, 17, 20, 22, 32, 25, 21, 20, 62, 33, 15, 18, 15, 17, 16, 18, 16, 18, 17, 62, 20, 23, 23, 48, 19, 22, 20, 29, 18, 55, 90, 62, 22, 17, 31, 18, 16, 39, 29, 26, 18, 76, 17, 12, 12, 26, 23, 23, 23, 18, 14, 16, 19, 20, 24, 16, 17, 12, 12, 26, 23, 23, 14, 24, 16, 20, 24, 16, 23, 18, 18, 18] 75


         1     -201127.4185             +nan
         2     -187549.2307      +13578.1878
         3     -185174.3546       +2374.8761
         4     -184284.7806        +889.5740
         5     -183981.0947        +303.6859
         6     -183828.3420        +152.7527
         7     -183684.8579        +143.4840
         8     -183620.2569         +64.6010
         9     -183595.5747         +24.6823
        10     -183578.7390         +16.8357
        11     -183558.2359         +20.5030
        12     -183544.5511         +13.6849
        13     -183533.6802         +10.8708
        14     -183525.3596          +8.3206
        15     -183518.1086          +7.2510
        16     -183514.0716          +4.0371
        17     -183508.3489          +5.7227
        18     -183503.8735          +4.4753
        19     -183500.3561          +3.5175
        20     -183493.7748          +6.5813
        21     -183456.4302         +37.3446
        22     -183445.6761         +10.7541
        23

(2020, 36) [29, 30, 15, 43, 30, 30, 14, 25, 24, 30, 35, 23, 17, 24, 21, 31, 22, 28, 29, 28, 19, 16, 16, 31, 24, 22, 19, 13, 21, 23, 30, 19, 32, 47, 18, 34, 26, 21, 10, 23, 29, 25, 21, 25, 25, 12, 27, 24, 37, 22, 26, 29, 17, 26, 24, 33, 36, 22, 21, 30, 31, 31, 22, 22, 26, 23, 36, 32, 23, 17, 32, 28, 21, 32, 21, 16, 25, 26, 35, 18] 80


         1     -211663.4078             +nan
         2     -198267.9560      +13395.4518
         3     -196269.0726       +1998.8834
         4     -195734.8810        +534.1916
         5     -195378.8148        +356.0663
         6     -195202.1334        +176.6814
         7     -195102.1161        +100.0173
         8     -195028.6159         +73.5001
         9     -195001.3046         +27.3114
        10     -194969.4793         +31.8253
        11     -194918.6741         +50.8052
        12     -194865.9483         +52.7258
        13     -194821.6840         +44.2643
        14     -194800.1977         +21.4863
        15     -194790.0401         +10.1576
        16     -194782.2965          +7.7435
        17     -194779.1484          +3.1481
        18     -194775.9613          +3.1871
        19     -194774.9361          +1.0252
        20     -194774.2010          +0.7351
        21     -194773.5629          +0.6381
        22     -194772.9822          +0.5807
        23

(3536, 36) [52, 43, 35, 55, 34, 55, 47, 51, 41, 49, 34, 58, 41, 41, 75, 30, 50, 53, 39, 42, 40, 42, 49, 40, 39, 36, 40, 41, 54, 47, 46, 55, 38, 49, 37, 35, 45, 39, 58, 42, 40, 43, 34, 31, 47, 48, 37, 58, 38, 37, 50, 41, 37, 43, 50, 49, 48, 44, 51, 32, 42, 51, 42, 37, 30, 50, 45, 35, 39, 55, 47, 45, 39, 40, 61, 56, 42, 40, 46, 39] 80


         1     -366055.0503             +nan
         2     -333502.9411      +32552.1092
         3     -328985.8096       +4517.1315
         4     -327813.6159       +1172.1937
         5     -327082.4470        +731.1688
         6     -326473.8937        +608.5534
         7     -326007.9192        +465.9744
         8     -325782.1956        +225.7237
         9     -325634.4880        +147.7076
        10     -325514.8303        +119.6577
        11     -325441.4115         +73.4188
        12     -325390.0395         +51.3720
        13     -325343.0646         +46.9748
        14     -325276.8430         +66.2216
        15     -325192.7062         +84.1368
        16     -325115.9349         +76.7713
        17     -325071.0340         +44.9010
        18     -325052.1386         +18.8954
        19     -325031.8893         +20.2492
        20     -324980.5655         +51.3238
        21     -324919.1578         +61.4077
        22     -324848.0403         +71.1175
        23

In [63]:
print("Testing")
dataset["test_nha"] = func.get_class_data(os.path.join("data_hmm", "test_nha"))
for true_cname in class_names:
    if true_cname[:4] == "test":
        result = 0
        length = 0
        for O in dataset[true_cname]:
            length+=1
            score = {cname: model.score(O, [len(O)]) for cname, model in models.items()}
            if true_cname[5:] == max(score.items(), key=operator.itemgetter(1))[0]:
                result = result + 1
            else: 
                print(true_cname, score, length, max(score.items(), key=operator.itemgetter(1))[0])
        print('Acc ' + true_cname[5:] , result/length, result, length)

Testing


Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance


Acc nguoi 1.0 20 20


Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance


Acc viet_nam 1.0 20 20


...

Acc nha 1.0 20 20


Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance


Acc cua 1.0 20 20


Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance


Acc khong 1.0 20 20


In [28]:
import pickle

In [64]:
with open("modelHMM.pkl", "wb") as file:
    pickle.dump(models, file)